In [1]:
import os
import sys
from pathlib import Path
sys.path.insert(1, os.path.realpath(os.path.pardir))
import torch

from models.mirasol import Mirasol, MirasolConfig, Franky
from models.vq_brain_per_channel import SoundStream, VAEConfig


In [2]:
x = torch.randn(2, 128, 256)


vae_config = VAEConfig(C=256, levels=(8, 8, 6, 5))
vq_vae = SoundStream(**vae_config.to_dict())

config = MirasolConfig(n_electrodes=256, n_registers=8, dim=64, window_size=128)
brain_model = Mirasol(config, vq_vae)



self.codebook_size 1920
self.downsample 8
Shape of the rope cache:  torch.Size([512, 8])
Full Mirasol model: number of parameters: 15.66M


In [3]:
x = torch.randn(2, 128, 256)
x[:, 64:] = 0

loss, features=brain_model(x)

In [4]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None

In [5]:
tokenizer = processor.tokenizer
whisper = {'decoder': model.model.decoder, 
           'proj_out': model.proj_out, 
           'tokenizer': tokenizer}


In [6]:
franky = Franky(brain_model=brain_model, llm_model=whisper) 

Full Franky: number of parameters: 45.25M


In [7]:
tokenizer.eos_token_id

50257

In [8]:
franky.proj_out

Linear(in_features=384, out_features=51865, bias=False)

In [9]:
loss = {'one': 2}
loss['three'] =  3




In [21]:
x = torch.randn(2, 128, 256)

txt = tokenizer('I live in Russia', return_tensors='pt')['input_ids']

loss2, _ = franky(x, txt)

In [22]:
loss_list = [loss, loss2]  # Assuming 'loss' is a dictionary and you have a list of similar dictionaries
average_loss = {key: sum(d[key] for d in loss_list) / len(loss_list) for key in loss_list[0]}


In [24]:
loss_list

[{'total_loss': tensor(15.8810, grad_fn=<AddBackward0>),
  'latent_loss': tensor(0.1429, grad_fn=<RsubBackward1>),
  'recon_loss': tensor(7.5791, grad_fn=<NllLossBackward0>),
  'txt_loss': tensor(8.1590, grad_fn=<NllLossBackward0>)},
 {'total_loss': tensor(15.8998, grad_fn=<AddBackward0>),
  'latent_loss': tensor(0.1504, grad_fn=<RsubBackward1>),
  'recon_loss': tensor(7.5791, grad_fn=<NllLossBackward0>),
  'txt_loss': tensor(8.1703, grad_fn=<NllLossBackward0>)}]

In [23]:
average_loss

{'total_loss': tensor(15.8904, grad_fn=<DivBackward0>),
 'latent_loss': tensor(0.1467, grad_fn=<DivBackward0>),
 'recon_loss': tensor(7.5791, grad_fn=<DivBackward0>),
 'txt_loss': tensor(8.1646, grad_fn=<DivBackward0>)}

In [11]:
loss, features.shape

({'one': 2, 'three': 3}, torch.Size([2, 120, 64]))

In [12]:
import torch
import torch.nn.functional as F

# Test data
B, T, D = 2, 5, 3  # Batch size, Time steps, Dimension
x = torch.randn(B, T, D)
y = torch.randn(B, T, D)

# Padding mask (1 where data is valid, 0 where it is padded)
is_padded = torch.tensor([
    [False, False, True, True, True],
    [False, False, True, True, True]
], dtype=torch.bool)

# Calculate cosine loss
loss = cosine_loss(x, y, is_padded)
print("Cosine Loss:", loss)

loss = cosine_loss(x[:, :2], y[:, :2], is_padded[:, :2])
print(loss)


NameError: name 'cosine_loss' is not defined

In [ ]:
import torch.nn.functional as F

x = torch.ones(2, 512)

torch.repeat_interleave(x, 2, dim=-1).shape


torch.Size([2, 1024])